In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ssd\\Desktop\\PRO\\End-to-End-MLOps-Developer-Salary-Predictions\\explore'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ssd\\Desktop\\PRO\\End-to-End-MLOps-Developer-Salary-Predictions'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from mlProject.utils import *

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from mlProject.custom_logger import logger


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-23 19:22:53,914: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2023-12-23 19:22:53,926: INFO: utils: yaml file: config\params.yaml loaded successfully]
[2023-12-23 19:22:53,929: INFO: utils: yaml file: config\schema.yaml loaded successfully]
[2023-12-23 19:22:53,931: INFO: utils: created directory at: artifacts]
[2023-12-23 19:22:53,934: INFO: utils: created directory at: artifacts/data_ingestion]


[2023-12-23 19:23:15,224: INFO: 601478533: artifacts/data_ingestion/data.zip download! with following info: 
Content-Type: application/zip
Content-Length: 20599404
Vary: Origin
Accept-Ranges: bytes
Content-Security-Policy: default-src 'self'; script-src 'none'
Last-Modified: Fri, 30 Jun 2023 14:10:18 GMT
Cache-Control: public, max-age=31536000, s-maxage=2592000
x-sanity-storage-adapter: gcs-default
X-Sanity-Md5: feaccac6de870f5c2cd9eb888a4af34c
ETag: feaccac6de870f5c2cd9eb888a4af34c
Content-Disposition: inline;filename="stack-overflow-developer-survey-2023.zip"
Date: Sat, 23 Dec 2023 18:22:52 GMT
strict-transport-security: max-age=63072000; includeSubDomains; preload
Via: 1.1 google
Alt-Svc: h3=":443"; ma=2592000,h3-29=":443"; ma=2592000
Connection: close

]
